In [1]:
from requests.adapters import HTTPAdapter

In [2]:
from requests.packages.urllib3.poolmanager import PoolManager
import ssl
import mechanicalsoup

In [3]:
class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1)

In [4]:
data = {
    'username': 'ballaneypranav',
    'password': 'Pza29kgNwLuyyz',
    'Login'   : 'Login'
}

In [5]:
browser = mechanicalsoup.StatefulBrowser()

In [6]:
browser.open("https://igem.org/Login2")

<Response [200]>

In [7]:
browser.select_form('form[method="post"]')

In [8]:
browser.get_current_form().print_summary()

<input name="return_to" type="hidden" value=""/>
<input name="username" size="20" type="text" value=""/>
<input name="password" size="20" type="password" value=""/>
<input class="submit" name="Login" type="submit" value="Login"/>
<input class="link_text" name="mail_password" type="submit" value="Email me a new password"/>


In [9]:
browser["username"] = data["username"]
browser["password"] = data["password"]

In [10]:
response = browser.submit_selected()

In [11]:
print(response.text)

<div style='width:500px;margin:0px auto;padding:20px;background-color:white;'>You have successfully logged into the iGEM web sites.</div>


In [12]:
team = 'BITSPilani-Goa_India'
page = ''
pageType = 'Team'
year = '2020'
url = 'https://' + year + '.igem.org/wiki/index.php?title=' + pageType + ':' + team + page + '&action=edit'
print(url)

https://2020.igem.org/wiki/index.php?title=Team:BITSPilani-Goa_India&action=edit


In [13]:
browser.open(url)

<Response [200]>

In [14]:
browser.select_form('form')

In [15]:
browser.get_current_form().print_summary()

<input id="wpAntispam" name="wpAntispam" type="text" value=""/>
<input name="wpSection" type="hidden" value=""/>
<input name="wpStarttime" type="hidden" value="20200624175800"/>
<input name="wpEdittime" type="hidden" value="20200624174745"/>
<input id="wpScrolltop" name="wpScrolltop" type="hidden" value=""/>
<input name="wpAutoSummary" type="hidden" value="d41d8cd98f00b204e9800998ecf8427e"/>
<input name="oldid" type="hidden" value="0"/>
<input name="format" type="hidden" value="text/x-wiki"/>
<input name="model" type="hidden" value="wikitext"/>
<textarea accesskey="," class="mw-ui-input" cols="80" dir="ltr" id="wpTextbox1" lang="en" name="wpTextbox1" rows="25" style="" tabindex="1">{{BITSPilani-Goa_India}}
{{BITSPilani-Goa_India/mainCSS}}

hello</textarea>
<input accesskey="b" class="mw-summary mw-ui-input" id="wpSummary" maxlength="200" name="wpSummary" size="60" spellcheck="true" tabindex="1" title="Enter a short summary [b]"/>
<input accesskey="i" id="wpMinoredit" name="wpMinoredit"